In [4]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("ajaykarthick/imdb-movie-reviews")

In [5]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['review', 'label'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['review', 'label'],
        num_rows: 10000
    })
})


In [6]:
for i in range(5):
    print(f"Example {i}")
    print("Label:", ds["train"][i]["label"])
    print("Review preview:")
    print(ds["train"][i]["review"])
    print("----")


Example 0
Label: 0
Review preview:
Ms Aparna Sen, the maker of Mr & Mrs Iyer, directs this movie about a young girl's struggle to cope with her debilitating condition.<br /><br />Meethi (Konkona Sen) has been an aloof kid ever since childhood and has shown signs of delusion, no one knows why. The dormant tendency however slips out of control, when the job assignment takes her to neighboring Bihar where she's raped by some political goons. The resulting trauma also leads to episodes of manic-depressive psychosis in addition to her schizophrenia. She careens out of control over the years, progressively getting worse and sinking deeper into her private 'world'.<br /><br />The juxtaposition of an 'unsettled' (divorced) elder sister and how her domineering ways make an already bad situation worse, is indicative of what a fine line there is between abnormal and *seemingly normal*. Ms Sen also makes an excellent commentary on the social alienation of such individuals. Social rehab is standard

In [7]:
#check label distribution(dataset is balanced or not)
from collections import Counter

labels = ds["train"]["label"]
Counter(labels)


Counter({0: 20000, 1: 20000})

In [8]:
import pandas as pd
import numpy as np

# Convert HuggingFace dataset to DataFrame for easier handling
train_df = pd.DataFrame(ds["train"])
test_df  = pd.DataFrame(ds["test"])

In [9]:
import re

def extract_style_features(df):
    features = pd.DataFrame()
    features['num_exclamation'] = df['review'].apply(lambda x: x.count('!'))
    features['num_question'] = df['review'].apply(lambda x: x.count('?'))
    features['num_ellipsis'] = df['review'].apply(lambda x: x.count('...') + x.count('…'))
    # Count all-caps words (length >= 2 to avoid single letters)
    features['num_all_caps_words'] = df['review'].apply(lambda x: len(re.findall(r'\b[A-Z]{2,}\b', x)))
    return features
    
train_style_features = extract_style_features(train_df)
test_style_features = extract_style_features(test_df)

In [10]:
print(train_style_features.head())

   num_exclamation  num_question  num_ellipsis  num_all_caps_words
0                1             1             0                   4
1                0             0             0                  15
2                1             0             0                   0
3                4             0             0                   1
4                0             0             0                   0


In [20]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from bs4 import BeautifulSoup
import html

stop_words = set(stopwords.words('english'))

def clean_text(text):

    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    # Remove literal <br>, <br />, <p>, etc.
    text = re.sub(r'<br\s*/?>', ' ', text, flags=re.IGNORECASE)
    text = re.sub(r'<p\s*/?>', ' ', text, flags=re.IGNORECASE)
    # Decode HTML entities (&amp;, &quot;)
    text = html.unescape(text)
    # Remove weird unicode characters
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    # Lowercase
    text = text.lower()
    # Remove punctuation and special characters
    text = re.sub(r'[^a-z\s]', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Optional: remove stopwords
    text_tokens = [w for w in text.split() if w not in stop_words]
    return ' '.join(text_tokens)

# Apply to train/test reviews
train_df['clean_review'] = train_df['review'].apply(clean_text)
test_df['clean_review'] = test_df['review'].apply(clean_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fatim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
# Show original and cleaned review side by side
compare_df = train_df[['review', 'clean_review']].head(5)  
pd.set_option('display.max_colwidth', None)  # show full text

compare_df


,review,clean_review
0,"Ms Aparna Sen, the maker of Mr & Mrs Iyer, directs this movie about a young girl's struggle to cope with her debilitating condition.<br /><br />Meethi (Konkona Sen) has been an aloof kid ever since childhood and has shown signs of delusion, no one knows why. The dormant tendency however slips out of control, when the job assignment takes her to neighboring Bihar where she's raped by some political goons. The resulting trauma also leads to episodes of manic-depressive psychosis in addition to her schizophrenia. She careens out of control over the years, progressively getting worse and sinking deeper into her private 'world'.<br /><br />The juxtaposition of an 'unsettled' (divorced) elder sister and how her domineering ways make an already bad situation worse, is indicative of what a fine line there is between abnormal and *seemingly normal*. Ms Sen also makes an excellent commentary on the social alienation of such individuals. Social rehab is standard therapy along with all the deadly mind-altering drugs. But what about the poor and the destitute, who're always left to fend for themselves and usually fall by the wayside?<br /><br />The romantic connection between Dr Kunal and Anu was unnecessary. Also the cafeteria scene where Dr Kunal explains to Anu how real their world really is to them, was redundant. Anu should already know all that. The English dialog is a bit awkward at times though the acting compensates for that. Konkona and Shabana prove that their reputation is every bit worth it. Waheeda, Rahul and Shefali play their limited roles very well. <br /><br />Extensive research seems to have been done about this illness, its very evident. But its not clear if MDP can coexist with schizophrenia in the same patient, side-by-side. Also in the early part, Dr Kunal recommends E.C.T (shock therapy) while invalidating the fact that it doesn't work for schizophrenics, only for extreme MDP with suicidal tendencies and other forms of bipolar disorder.<br /><br />The ending of the remarkable story is suggestive of an unknown solution (maybe no solution). The movie could have ended on a nicer note, since worldwide the mentally ill can and do lead balanced and fruitful if not very fulfilling, lives under good medical care.<br /><br />Nonetheless, its an excellent film made with extreme sensitivity to the subject. HATS OFF to Ms Sen! No one in India could've done it better.",ms aparna sen maker mr mrs iyer directs movie young girls struggle cope debilitating conditionmeethi konkona sen aloof kid ever since childhood shown signs delusion one knows dormant tendency however slips control job assignment takes neighboring bihar shes raped political goons resulting trauma also leads episodes manicdepressive psychosis addition schizophrenia careens control years progressively getting worse sinking deeper private worldthe juxtaposition unsettled divorced elder sister domineering ways make already bad situation worse indicative fine line abnormal seemingly normal ms sen also makes excellent commentary social alienation individuals social rehab standard therapy along deadly mindaltering drugs poor destitute whore always left fend usually fall waysidethe romantic connection dr kunal anu unnecessary also cafeteria scene dr kunal explains anu real world really redundant anu already know english dialog bit awkward times though acting compensates konkona shabana prove reputation every bit worth waheeda rahul shefali play limited roles well extensive research seems done illness evident clear mdp coexist schizophrenia patient sidebyside also early part dr kunal recommends ect shock therapy invalidating fact doesnt work schizophrenics extreme mdp suicidal tendencies forms bipolar disorderthe ending remarkable story suggestive unknown solution maybe solution movie could ended nicer note since worldwide mentally ill lead balanced fruitful fulfilling lives good medical carenonetheless excellent film made extreme sensitivity subject

In [23]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['clean_review'])
X_test_tfidf  = tfidf_vectorizer.transform(test_df['clean_review'])

print("TF-IDF shape:", X_train_tfidf.shape)


TF-IDF shape: (40000, 5000)


In [24]:
from scipy.sparse import hstack



# Combine TF-IDF (sparse) + style features (dense)
X_train_combined = hstack([X_train_tfidf, train_style_features.values])
X_test_combined  = hstack([X_test_tfidf, test_style_features.values])

# Labels
y_train = train_df['label'].values
y_test  = test_df['label'].values

print("Combined feature matrix shape:", X_train_combined.shape)


Combined feature matrix shape: (40000, 5004)


In [25]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Create the model
nb_model = MultinomialNB()

# Train on combined features
nb_model.fit(X_train_combined, y_train)

# Predict on test set
y_pred = nb_model.predict(X_test_combined)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.8186

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.86      0.83      5000
           1       0.85      0.78      0.81      5000

    accuracy                           0.82     10000
   macro avg       0.82      0.82      0.82     10000
weighted avg       0.82      0.82      0.82     10000


Confusion Matrix:
 [[4291  709]
 [1105 3895]]


In [26]:
import pandas as pd

# Create a DataFrame for test set with predictions
test_results = pd.DataFrame({
    'review': test_df['review'],
    'actual': y_test,
    'predicted': y_pred
})

# Filter only wrong predictions
wrong_preds = test_results[test_results['actual'] != test_results['predicted']]

# Show first 5 wrong predictions
for i, row in wrong_preds.head(10).iterrows():
    print(f"Actual: {row['actual']} | Predicted: {row['predicted']}")
    print(f"Review preview: {row['review'][:300]}")  # show first 300 characters
    print("----")


Actual: 1 | Predicted: 0
Review preview: For the most part, 'Michael' is a disaster  ten minutes of charm and ninety's worth of missteps.<br /><br />Travolta and MacDowell do their best, frequently rising above Nora Ephron's numbingly banal script. But the film moves like a snail. And even within its fantasy context, the characters behave
----
Actual: 1 | Predicted: 0
Review preview: if you're a sucker for corny movies and are looking to see something you don't need to pay close attention to, this might be worth watching. the story itself is very unrealistic. the dialogue is also not very believable. it is doubtful you will find yourself relating to any of these characters becau
----
Actual: 0 | Predicted: 1
Review preview: As a long time fan of Peter O'Donnell's greatest creation, I watched this film on DVD with no great hopes of enjoyment; indeed I expected to be reaching in disgust for the remote control within fifteen minutes. But instead I thoroughly enjoyed this production, and 